현재 상태:

Tumor 디렉토리:

    - SRR30510137: ✅ 완료

    - SRR30510151: ✅ 완료

    - SRR30510181: ✅ 완료

Normal 디렉토리:

    - SRR28602760: ✅ 완료

    - SRR28602761: ✅ 완료

    - SRR28602767: ✅ 완료

각 디렉토리 별로 아래 단계가 완료된 상태입니다.

- fastqc (품질 체크)

- trim_galore (트리밍)

- STAR (정렬 및 BAM 파일 생성)

- featureCounts (유전자 발현량 정량화, counts.txt 생성)

In [9]:
import pandas as pd
import glob
import os

tumor_paths = glob.glob('./Tumor/*/counts.txt')
normal_paths = glob.glob('./Normal/*/counts.txt')

usecols = ['Geneid', 'SRR']

counts_tumor = []
for path in tumor_paths:
    sample_name = os.path.basename(os.path.dirname(path))
    df = pd.read_csv(path, sep='\t', comment='#', usecols=[0, 6], names=['Geneid', sample_name], header=0)
    counts_tumor.append(df.set_index('Geneid'))

counts_tumor_df = pd.concat(counts_tumor, axis=1)

counts_normal = []
for path in normal_paths:
    sample_name = os.path.basename(os.path.dirname(path))
    df = pd.read_csv(path, sep='\t', comment='#', usecols=[0, 6], names=['Geneid', sample_name], header=0)
    counts_normal.append(df.set_index('Geneid'))

counts_normal_df = pd.concat(counts_normal, axis=1)

combined_counts_df = pd.concat([counts_tumor_df, counts_normal_df], axis=1)
combined_counts_df.columns.name = 'Sample'

print(combined_counts_df.head())

Sample           SRR30510137  SRR30510151  SRR30510181  SRR28602760  \
Geneid                                                                
ENSG00000279928            6            0            2            2   
ENSG00000228037            0            0            7           15   
ENSG00000142611         2403          735          168         1128   
ENSG00000284616            0            0            0            0   
ENSG00000157911         1369          537          889          805   

Sample           SRR28602761  SRR28602767  
Geneid                                     
ENSG00000279928            2            6  
ENSG00000228037            8            6  
ENSG00000142611         1247          504  
ENSG00000284616            0            0  
ENSG00000157911         1177         1301  


In [10]:
combined_counts_df.columns = [
    'Tumor_SRR30510137',
    'Tumor_SRR30510151',
    'Tumor_SRR30510181',
    'Normal_SRR28602760',
    'Normal_SRR28602761',
    'Normal_SRR28602767'
]

In [11]:
combined_counts_df.head()

,Tumor_SRR30510137,Tumor_SRR30510151,Tumor_SRR30510181,Normal_SRR28602760,Normal_SRR28602761,Normal_SRR28602767
Geneid,,,,,,
ENSG00000279928,6,0,2,2,2,6
ENSG00000228037,0,0,7,15,8,6
ENSG00000142611,2403,735,168,1128,1247,504
ENSG00000284616,0,0,0,0,0,0
ENSG00000157911,1369,537,889,805,1177,1301
